In [1]:
from tensorflow.keras.models import load_model
from functions import margin_loss_fun_factory,DataGenerator,PredictGenerator,\
                        l2_distance_np,top_k,loop_distance,MAP,generate_feature
import pickle
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
from utility.albumentations_helper import create_transform
import time
import tensorflow as tf
import numpy as np
from functools import partial

In [2]:
color = False
#FFA_size = 4

Load model

In [3]:
feature_model = load_model('Models/feature_model_DenseNet_greyscale.h5')

In [4]:
#feature_model = load_model('Models/feature_l2_distance_margin_loss_fun_factory(0.1,5).h5', \
#                            custom_objects={'loss': margin_loss_fun_factory(0.1,5)})

In [4]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

In [5]:
aug = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform = create_transform(aug)    

In [6]:
aggFuns = [partial(np.quantile,q=pct,axis=(1,2)) for pct in [0.1,0.25,0.5]] + [partial(np.mean,axis=(1,2))]    

In [7]:
FFA_sizes = [4,8]
results = []

In [ ]:
for FFA_size in FFA_sizes:
    feature_train = generate_feature(Ids_train,transform,FFA_size,color,feature_model)
    feature_val = generate_feature(Ids_val,transform,FFA_size,color,feature_model)
    for aggFun in aggFuns:
        predicts = loop_distance(feature_train,feature_val,l2_distance_np,aggFun)
        mapping_dict = dict(zip(Ids_train.Id.values,Ids_train.index.values))
        labels = Ids_val.Id.map(mapping_dict)
        score = MAP(labels,predicts)
        print([FFA_size,aggFun.keywords,score])
        results.append([FFA_size,aggFun.keywords,score])

In [ ]:
# outputs 
[4, {'q': 0.1, 'axis': (1, 2)}, 0.0707482993197279]
[4, {'q': 0.25, 'axis': (1, 2)}, 0.05867346938775512]
[4, {'q': 0.5, 'axis': (1, 2)}, 0.0367965367965368]
[4, {'axis': (1, 2)}, 0.03235930735930736]
[8, {'q': 0.1, 'axis': (1, 2)}, 0.06657390228818802]
[8, {'q': 0.25, 'axis': (1, 2)}, 0.05171614100185529]
[8, {'q': 0.5, 'axis': (1, 2)}, 0.035219542362399506]
[8, {'axis': (1, 2)}, 0.029870129870129873]

In [6]:
aggFuns = [partial(np.quantile,q=pct,axis=(1,2)) for pct in [0.1,0.25]] + [partial(np.min,axis=(1,2))]    

In [7]:
FFA_sizes = [4,8]
results = []

In [8]:
for FFA_size in FFA_sizes:
    feature_train = generate_feature(Ids_train,transform,FFA_size,color,feature_model)
    feature_val = generate_feature(Ids_val,transform,FFA_size,color,feature_model)
    for aggFun in aggFuns:
        predicts = loop_distance(feature_train,feature_val,l2_distance_np,aggFun)
        mapping_dict = dict(zip(Ids_train.Id.values,Ids_train.index.values))
        labels = Ids_val.Id.map(mapping_dict)
        score = MAP(labels,predicts)
        print([FFA_size,aggFun.keywords,score])
        results.append([FFA_size,aggFun.keywords,score])

[4, {'q': 0.1, 'axis': (1, 2)}, 0.06901669758812615]
[4, {'q': 0.25, 'axis': (1, 2)}, 0.061750154607297464]
[4, {'axis': (1, 2)}, 0.06524427952999381]
[8, {'q': 0.1, 'axis': (1, 2)}, 0.07121212121212123]
[8, {'q': 0.25, 'axis': (1, 2)}, 0.051978973407544836]
[8, {'axis': (1, 2)}, 0.06589363017934448]


TTA = 4

Models/feature_model_DenseNet_greyscale.h5

In [23]:
feature_train = generate_feature(Ids_train,transform,FFA_size,color,feature_model)
feature_val = generate_feature(Ids_val,transform,FFA_size,color,feature_model)
predicts = loop_distance(feature_train,feature_val,l2_distance_np)
mapping_dict = dict(zip(Ids_train.Id.values,Ids_train.index.values))
labels = Ids_val.Id.map(mapping_dict)
score = MAP(labels,predicts)
print(score)

0.03713667285095857


In [25]:
# change mean to median
def loop_distance(feature_train,feature_val,distanceFun,returnValue=False,k=5):
    feature_train = feature_train[:,:,np.newaxis,:]
    index_list = []
    if returnValue:
        value_list = []

    for feature in feature_val:
        feature = feature[np.newaxis,np.newaxis,:,:]
        d = np.median(distanceFun(feature_train,feature),(1,2))
        if returnValue:
            index,value = top_k(d,k=k,returnValue=returnValue)
            index_list.append(index)
            value_list.append(value)
        else:
            index = top_k(d,k=k,returnValue=returnValue)
            index_list.append(index)

    if returnValue:
        return np.array(index_list),np.array(value_list)
    else:
        return np.array(index_list)

In [26]:
predicts = loop_distance(feature_train,feature_val,l2_distance_np)
mapping_dict = dict(zip(Ids_train.Id.values,Ids_train.index.values))
labels = Ids_val.Id.map(mapping_dict)
score = MAP(labels,predicts)
print(score)

0.04282622139764997


In [27]:
# change mean to 0.25 quantile
def loop_distance(feature_train,feature_val,distanceFun,returnValue=False,k=5):
    feature_train = feature_train[:,:,np.newaxis,:]
    index_list = []
    if returnValue:
        value_list = []

    for feature in feature_val:
        feature = feature[np.newaxis,np.newaxis,:,:]
        d = np.quantile(distanceFun(feature_train,feature),0.25,(1,2))
        if returnValue:
            index,value = top_k(d,k=k,returnValue=returnValue)
            index_list.append(index)
            value_list.append(value)
        else:
            index = top_k(d,k=k,returnValue=returnValue)
            index_list.append(index)

    if returnValue:
        return np.array(index_list),np.array(value_list)
    else:
        return np.array(index_list)

In [28]:
predicts = loop_distance(feature_train,feature_val,l2_distance_np)
mapping_dict = dict(zip(Ids_train.Id.values,Ids_train.index.values))
labels = Ids_val.Id.map(mapping_dict)
score = MAP(labels,predicts)
print(score)

0.059863945578231305


In [29]:
# change mean to 0.75 quantile
def loop_distance(feature_train,feature_val,distanceFun,returnValue=False,k=5):
    feature_train = feature_train[:,:,np.newaxis,:]
    index_list = []
    if returnValue:
        value_list = []

    for feature in feature_val:
        feature = feature[np.newaxis,np.newaxis,:,:]
        d = np.quantile(distanceFun(feature_train,feature),0.75,(1,2))
        if returnValue:
            index,value = top_k(d,k=k,returnValue=returnValue)
            index_list.append(index)
            value_list.append(value)
        else:
            index = top_k(d,k=k,returnValue=returnValue)
            index_list.append(index)

    if returnValue:
        return np.array(index_list),np.array(value_list)
    else:
        return np.array(index_list)

In [30]:
predicts = loop_distance(feature_train,feature_val,l2_distance_np)
mapping_dict = dict(zip(Ids_train.Id.values,Ids_train.index.values))
labels = Ids_val.Id.map(mapping_dict)
score = MAP(labels,predicts)
print(score)

0.03073593073593074


In [39]:
# change mean to 0.1 quantile
def loop_distance(feature_train,feature_val,distanceFun,returnValue=False,k=5):
    feature_train = feature_train[:,:,np.newaxis,:]
    index_list = []
    if returnValue:
        value_list = []

    for feature in feature_val:
        feature = feature[np.newaxis,np.newaxis,:,:]
        d = np.quantile(distanceFun(feature_train,feature),0.1,(1,2))
        if returnValue:
            index,value = top_k(d,k=k,returnValue=returnValue)
            index_list.append(index)
            value_list.append(value)
        else:
            index = top_k(d,k=k,returnValue=returnValue)
            index_list.append(index)

    if returnValue:
        return np.array(index_list),np.array(value_list)
    else:
        return np.array(index_list)

In [40]:
predicts = loop_distance(feature_train,feature_val,l2_distance_np)
mapping_dict = dict(zip(Ids_train.Id.values,Ids_train.index.values))
labels = Ids_val.Id.map(mapping_dict)
score = MAP(labels,predicts)
print(score)

0.07564935064935066
